In [1]:
import numpy as np
np.random.seed(42)

import pandas as pd
from sklearn.externals import joblib

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import os
print(os.listdir("input/"))

from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
import xgboost as xgb

import seaborn as sns

print("Imports Complete\n")

Using TensorFlow backend.


['Y_train.txt', 'X_train.txt', 'activity_labels.txt', 'sample_submission.txt', 'X_test.txt', 'features.txt', 'features_info.txt', 'README.txt']
Imports Complete



In [2]:
print("loading X")
X = pd.read_csv('input/X_train.txt', sep='\s+',header=None)
X_original=X
print(X.head())
print(X.shape)

print("loading XFinal")
final_X = pd.read_csv('input/X_test.txt', sep='\s+',header=None)
final_X_original=final_X


print("loading Y")
Y = pd.read_csv('input/Y_train.txt', sep='\s+',header=None)
Y = Y.astype(int)
print(Y.head())
print(Y.shape)

print("Converting y to Categorical Vector")
Y_original = Y
#original Y saved in Y_original
Y = keras.utils.to_categorical(Y,dtype='int32')
print(Y[:8])
print(Y.shape)

print("Data Loaded")
X_original=X
X = X.values
final_X = final_X.values


loading X
        0         1         2         3         4         5         6    \
0  0.271925 -0.017309 -0.106774 -0.982197 -0.986899 -0.967111 -0.983637   
1  0.335256  0.019098 -0.055751  0.080347  0.096305 -0.136543 -0.049000   
2  0.131517 -0.115616 -0.208167 -0.246124 -0.274395 -0.059285 -0.326047   
3  0.279240 -0.018476 -0.105920 -0.984630 -0.966410 -0.952940 -0.985880   
4  0.285120  0.006762 -0.087324 -0.417612 -0.093410 -0.654948 -0.425524   

        7         8         9      ...          551       552       553  \
0 -0.986711 -0.963578 -0.925383    ...     0.153164 -0.438706 -0.767483   
1 -0.044129 -0.248500  0.483926    ...    -0.144744 -0.029993 -0.471274   
2 -0.310820 -0.023570  0.041278    ...     0.376784 -0.588954 -0.786680   
3 -0.972560 -0.950020 -0.930450    ...     0.218200 -0.213440 -0.596880   
4 -0.090418 -0.646486 -0.295091    ...     0.205700 -0.397920 -0.739947   

        554       555       556       557       558       559       560  
0  0.138323  0

In [19]:
print("Splitting Train-Test")
trainX, testX, trainY, testY = train_test_split(X_original, Y_original, test_size=0.2)
print ("trainX shape = ",trainX.shape,"trainY shape = ", trainY.shape)
print ("testX shape = ", testX.shape, "testY shape = ", testY.shape)
print( "finalX shape = ", final_X_original.shape)
print("Finding Feature Importances\n")
acc=0.0
md,ne=3,250
f_model = XGBClassifier(learning_rate=0.8,max_depth=md, n_estimators=ne, n_jobs=3, verbosity=2, min_child_weight=1,subsample=1, random_state=0)
f_model.fit(trainX, np.ravel(trainY))
predY = f_model.predict(testX)
predY=predY.astype('int32')
testY = testY.astype('int32')

acc = sklearn.metrics.accuracy_score(testY,predY,normalize='True') * 100
print("md=",md,"ne=",ne,"Accuracy = ",acc, "%\n")


if (acc)>97.0:
    joblib.dump(f_model, str(md)+str(ne)+'_'+str(acc)+".joblib.dat")
    f_model = joblib.load(str(md)+str(ne)+'_'+str(acc)+".joblib.dat")
#             final_predY = f_model.predict(final_X_original)
#             #final_predY = np.argmax(final_predY, axis = 1)
#             final_predY=final_predY.astype('int32')
#             for i in range(len(final_predY)):
#                 print(str(i)+","+str(final_predY[i]))

Splitting Train-Test
trainX shape =  (7700, 561) trainY shape =  (7700, 1)
testX shape =  (1925, 561) testY shape =  (1925, 1)
finalX shape =  (6418, 561)
Finding Feature Importances

md= 3 ne= 250 Accuracy =  96.72727272727273 %



In [ ]:
print("Splitting Train-Test")
trainX, testX, trainY, testY = train_test_split(X_original, Y_original, test_size=0.2)
print ("trainX shape = ",trainX.shape,"trainY shape = ", trainY.shape)
print ("testX shape = ", testX.shape, "testY shape = ", testY.shape)
print( "finalX shape = ", final_X_original.shape)
print("Finding Feature Importances\n")
acc=0.0
for md in range(7,101,5):
    for ne in range(150,2001,200):
        print("Fitting\n")
        # fit model no training data
        f_model = XGBClassifier(max_depth=md, n_estimators=ne, n_jobs=3, verbosity=2, min_child_weight=1,subsample=1, random_state=0)
        f_model.fit(trainX, np.ravel(trainY))
        # plot feature importance
        #plot_importance(f_model)
        #pyplot.show()
        # plot feature importance using built-in function
        predY = f_model.predict(testX)
        #predY = np.argmax(predY, axis = 1)
        predY=predY.astype('int32')
        #print("Predictions : ")
        #print("shape = " , predY.shape)
        #print(predY[:8])
        #print("Truth : ")
        #testY = np.argmax(testY, axis = 1)
        testY = testY.astype('int32')
        #print("shape = " , testY.shape)
        #print(testY[:8])

        acc = sklearn.metrics.accuracy_score(testY,predY,normalize='True') * 100
        print("md=",md,"ne=",ne,"Accuracy = ",acc, "%\n")
        
        
        if (acc)>97.0:
            joblib.dump(f_model, str(md)+'_'+str(ne)+'_'+str(acc)+".joblib.dat")
#             f_model = joblib.load(str(md)+str(ne)+'_'+str(acc)+".joblib.dat")
#             final_predY = f_model.predict(final_X_original)
#             #final_predY = np.argmax(final_predY, axis = 1)
#             final_predY=final_predY.astype('int32')
#             for i in range(len(final_predY)):
#                 print(str(i)+","+str(final_predY[i]))

Splitting Train-Test
trainX shape =  (7700, 561) trainY shape =  (7700, 1)
testX shape =  (1925, 561) testY shape =  (1925, 1)
finalX shape =  (6418, 561)
Finding Feature Importances

Fitting

md= 7 ne= 150 Accuracy =  95.84415584415584 %

Fitting

md= 7 ne= 350 Accuracy =  95.8961038961039 %

Fitting

md= 7 ne= 550 Accuracy =  96.05194805194806 %

Fitting

md= 7 ne= 750 Accuracy =  96.0 %

Fitting

md= 7 ne= 950 Accuracy =  96.15584415584416 %

Fitting

md= 7 ne= 1150 Accuracy =  96.20779220779221 %

Fitting

md= 7 ne= 1350 Accuracy =  96.15584415584416 %

Fitting

md= 7 ne= 1550 Accuracy =  96.25974025974025 %

Fitting

md= 7 ne= 1750 Accuracy =  96.25974025974025 %

Fitting

md= 7 ne= 1950 Accuracy =  96.25974025974025 %

Fitting

md= 12 ne= 150 Accuracy =  95.84415584415584 %

Fitting

md= 12 ne= 350 Accuracy =  96.05194805194806 %

Fitting

md= 12 ne= 550 Accuracy =  95.8961038961039 %

Fitting

md= 12 ne= 750 Accuracy =  95.8961038961039 %

Fitting

md= 12 ne= 950 Accuracy =  96.

In [4]:
f_model = joblib.load("/home/harsh/Desktop/Human Activity Recognition/kernel/4200_97.55844155844156.joblib.dat")
final_predY = f_model.predict(final_X_original)
#final_predY = np.argmax(final_predY, axis = 1)
final_predY=final_predY.astype('int32')
for i in range(len(final_predY)):
    print(str(i)+","+str(final_predY[i]))

/home/harsh/anaconda3/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


0,3
1,2
2,1
3,4
4,3
5,1
6,4
7,3
8,6
9,6
10,1
11,5
12,1
13,5
14,6
15,5
16,6
17,4
18,4
19,1
20,1
21,6
22,1
23,1
24,1
25,2
26,6
27,6
28,2
29,2
30,2
31,3
32,6
33,6
34,4
35,1
36,6
37,3
38,3
39,6
40,5
41,5
42,4
43,4
44,5
45,5
46,4
47,5
48,2
49,4
50,5
51,1
52,2
53,6
54,3
55,1
56,5
57,4
58,6
59,2
60,1
61,4
62,1
63,3
64,1
65,6
66,5
67,6
68,3
69,1
70,5
71,6
72,4
73,5
74,6
75,4
76,2
77,5
78,1
79,3
80,4
81,5
82,4
83,3
84,2
85,6
86,5
87,3
88,2
89,4
90,6
91,3
92,4
93,2
94,5
95,5
96,1
97,1
98,5
99,3
100,6
101,6
102,1
103,5
104,5
105,6
106,6
107,4
108,4
109,4
110,1
111,6
112,5
113,5
114,1
115,4
116,2
117,4
118,2
119,6
120,3
121,2
122,6
123,5
124,6
125,1
126,4
127,4
128,6
129,4
130,6
131,1
132,6
133,2
134,4
135,5
136,1
137,4
138,1
139,6
140,1
141,5
142,4
143,1
144,4
145,3
146,1
147,3
148,4
149,6
150,5
151,1
152,3
153,1
154,5
155,2
156,6
157,2
158,4
159,1
160,5
161,1
162,2
163,6
164,4
165,4
166,1
167,2
168,1
169,3
170,5
171,5
172,6
173,3
174,4
175,2
176,2
177,6
178,6
179,6
180,4
181,2
182,4
183,3
184,2


2449,6
2450,6
2451,5
2452,3
2453,3
2454,5
2455,1
2456,1
2457,4
2458,3
2459,4
2460,6
2461,4
2462,2
2463,2
2464,4
2465,3
2466,6
2467,4
2468,1
2469,1
2470,3
2471,1
2472,5
2473,1
2474,2
2475,1
2476,2
2477,2
2478,4
2479,6
2480,5
2481,3
2482,1
2483,4
2484,1
2485,4
2486,5
2487,2
2488,2
2489,1
2490,4
2491,1
2492,1
2493,1
2494,1
2495,5
2496,2
2497,3
2498,2
2499,6
2500,1
2501,3
2502,4
2503,6
2504,6
2505,4
2506,1
2507,1
2508,6
2509,6
2510,5
2511,4
2512,6
2513,1
2514,6
2515,4
2516,3
2517,1
2518,3
2519,2
2520,3
2521,1
2522,3
2523,3
2524,5
2525,6
2526,6
2527,2
2528,1
2529,5
2530,4
2531,6
2532,5
2533,4
2534,5
2535,2
2536,2
2537,2
2538,5
2539,1
2540,5
2541,6
2542,6
2543,6
2544,1
2545,2
2546,4
2547,3
2548,5
2549,1
2550,4
2551,1
2552,5
2553,2
2554,6
2555,3
2556,3
2557,4
2558,3
2559,4
2560,1
2561,4
2562,1
2563,6
2564,6
2565,4
2566,3
2567,1
2568,5
2569,6
2570,2
2571,4
2572,2
2573,5
2574,1
2575,6
2576,2
2577,6
2578,5
2579,5
2580,2
2581,6
2582,3
2583,2
2584,6
2585,6
2586,5
2587,1
2588,3
2589,1
2590,3
2591,1

4948,5
4949,1
4950,5
4951,4
4952,5
4953,6
4954,3
4955,5
4956,2
4957,2
4958,3
4959,2
4960,3
4961,3
4962,5
4963,4
4964,3
4965,4
4966,2
4967,1
4968,3
4969,3
4970,6
4971,4
4972,2
4973,1
4974,2
4975,3
4976,2
4977,4
4978,3
4979,2
4980,5
4981,2
4982,6
4983,2
4984,5
4985,2
4986,4
4987,4
4988,5
4989,2
4990,4
4991,5
4992,4
4993,3
4994,3
4995,2
4996,5
4997,3
4998,4
4999,3
5000,5
5001,5
5002,6
5003,5
5004,1
5005,1
5006,6
5007,2
5008,1
5009,1
5010,3
5011,4
5012,1
5013,4
5014,1
5015,2
5016,6
5017,5
5018,1
5019,2
5020,4
5021,1
5022,2
5023,2
5024,3
5025,1
5026,4
5027,5
5028,6
5029,1
5030,3
5031,1
5032,2
5033,5
5034,2
5035,4
5036,3
5037,1
5038,6
5039,2
5040,1
5041,2
5042,3
5043,3
5044,4
5045,4
5046,4
5047,1
5048,1
5049,3
5050,1
5051,5
5052,5
5053,1
5054,3
5055,6
5056,5
5057,5
5058,6
5059,4
5060,4
5061,3
5062,2
5063,2
5064,6
5065,6
5066,5
5067,5
5068,1
5069,5
5070,6
5071,5
5072,3
5073,5
5074,6
5075,1
5076,6
5077,4
5078,3
5079,5
5080,4
5081,4
5082,5
5083,5
5084,3
5085,2
5086,6
5087,6
5088,4
5089,6
5090,4